In [1]:
# !!!IMPORTANT!!! Don't run this anymore, we need the old results, without the changed .loading implementation.

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION' ] = 'python'

In [2]:
from lass.log_handling import LogLoaderArgs, LogLoader
from lass.data.loading import get_query_type
import lass.data


args = LogLoaderArgs(
    logdir="../artifacts/logs",
    tasks="paper-full",
    model_families=["BIG-G T=0"],
    model_sizes=["128b"],
    # model_sizes=["2m"],
    shots=[3],
    query_types=["multiple_choice", "scoring", "generative"],
)
loader = LogLoader(args)

In [3]:
data = lass.data.loading.load(args)
data = lass.data.wrangling.wrangle(
    data,
    include_model_in_input=False,
    include_n_targets_in_input=False,
    filter_bad_tasks=False,
)

In [4]:
data_filtered = lass.data.wrangling.wrangle(
    data,
    include_model_in_input=False,
    include_n_targets_in_input=False,
    filter_bad_tasks=True,
)

In [5]:
print(data.task.nunique())

161


In [6]:
data.groupby("query_type").task.nunique()

query_type
multiple_choice       118
scoring_generative     53
Name: task, dtype: int64

In [7]:
# This should be empty, but it is not! 10 tasks with duplicate queries (and thus instances)
import pandas as pd
pd.set_option('display.max_rows', None)
n_queries = data.groupby(["task"]).query_type.nunique()
n_queries = n_queries[n_queries > 1]
n_queries

task
checkmate_in_one            2
color                       2
emoji_movie                 2
geometric_shapes            2
kanji_ascii                 2
minute_mysteries_qa         2
penguins_in_a_table         2
similarities_abstraction    2
strategyqa                  2
unit_conversion             2
Name: query_type, dtype: int64

In [8]:
# Check which tasks are in the filtered data
problem_tasks = set(n_queries.reset_index().task.unique()).intersection(data_filtered.task.unique())
problem_tasks

{'emoji_movie', 'penguins_in_a_table', 'unit_conversion'}

In [9]:
# How many instances of these are actually duplicate?
print(f"{data_filtered.task.isin(n_queries.index).sum() / 2} potentially duplicate instances")

761.0 potentially duplicate instances


In [10]:
# Which of the problem tasks have over 500 instances? That they would be displayed individually in the paper.
data_filtered.query("task in @problem_tasks").groupby("task").size() / 2

task
emoji_movie            100.0
penguins_in_a_table    149.0
unit_conversion        512.0
dtype: float64